In [ ]:
import holoviews as hv
from holoviews import opts
from PIL import Image
import numpy as np
from holoviews import streams
from holoviews.streams import Pipe, Buffer
from holoviews.plotting.links import DataLink
import pandas as pd
hv.extension('bokeh')

In [ ]:
source_path = '../data/dario1.jpg'
source_img = Image.open(source_path)
source_img

In [ ]:
moving_path = '../data/dario2.jpg'
moving_img = Image.open(moving_path)
moving_img

In [ ]:
points_source = hv.Points([], vdims='color')
points_source_stream = streams.PointDraw(data=points_source.columns(), num_objects=10, source=points_source, empty_value='black')

source_image = hv.RGB(np.array(source_img), bounds=(0, 0, source_img.width, source_img.height))

source_table = hv.Table(points_source, ['x', 'y'], 'color')
DataLink(points_source, source_table)

static = ((source_image * points_source) + source_table).opts(
    opts.Points(active_tools=['point_draw'], color='color', show_grid=True,
                marker='s', size=10, tools=['hover', 'crosshair', 'undo']),
    opts.Table(editable=True)
)


moving_image = hv.RGB(np.array(moving_img), bounds=(0, 0, moving_img.width, moving_img.height))
moving_points_source = hv.Points([], vdims='color')
moving_points_source_stream = streams.PointDraw(data=moving_points_source.columns(), num_objects=10, source=moving_points_source, empty_value='black')

moving_table = hv.Table(moving_points_source, ['x', 'y'], 'color')
DataLink(moving_points_source, moving_table)

moving = ((moving_image * moving_points_source) + moving_table).opts(
    opts.Points(active_tools=['point_draw'], color='color', 
                marker='s', size=10, tools=['hover', 'crosshair']),
    opts.Table(editable=True)
)



layout = (static + moving).opts(merge_tools=False, shared_axes=False)
layout.cols(2)


In [ ]:
import cv2
import numpy as np
from pprint import pprint

In [ ]:
pprint(cv2.getBuildInformation())

In [ ]:
points_source_stream.contents,moving_points_source_stream.contents

In [ ]:
fixed_x = np.array(points_source_stream.data['x'])
fixed_y = np.array(points_source_stream.data['y'])
fixed = np.column_stack([fixed_x,fixed_y]).astype(np.int).reshape(1, -1, 2)
                       
moved_x = np.array(moving_points_source_stream.data['x'])
moved_y = np.array(moving_points_source_stream.data['y'])
moved = np.column_stack([moved_x,moved_y]).astype(np.int).reshape(1, -1, 2)

In [ ]:
fixed, moved

In [ ]:
matches = []
for i in range(fixed.shape[1]):
    matches.append(cv2.DMatch(i,i,0))
matches

In [ ]:
si_ = np.array(source_img)
mi_ = np.array(moving_img)
wi_ = np.ones_like(si_)

In [ ]:
si_.shape, mi_.shape, wi_.shape

In [ ]:
tps = cv2.createThinPlateSplineShapeTransformer()
tps.estimateTransformation(fixed, moved, matches)

In [ ]:
tps.warpImage(si_, wi_)

In [ ]:
hv.RGB(si_, bounds=(0, 0, moving_img.width, moving_img.height)) + hv.RGB(wi_, bounds=(0, 0, moving_img.width, moving_img.height))